In [1]:
import cv2
import numpy as np
import os
import glob

# Defining the dimensions of checkerboard
CHECKERBOARD = (11,8)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 


# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0]*CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

# Extracting path of individual image stored in a given directory
images = glob.glob('./Q1_Image/*.bmp')
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH+
    	cv2.CALIB_CB_FAST_CHECK+cv2.CALIB_CB_NORMALIZE_IMAGE)
    
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        
        imgpoints.append(corners2)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2,ret)
        img = cv2.resize(img,(512,512))
    cv2.imshow('img',img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

#print(objpoints)
#print("--"*10)
#print(imgpoints)

"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("size of dist：",np.shape(dist))
# cv2.calibrateCamera 使用 rotation vector 和 translation vector 來表示 extrinsic parameters
print("rvecs : \n")
print(rvecs)
print("size of rvecs：",np.shape(rvecs))
print("tvecs : \n")
print(tvecs)
print("size of tvecs：",np.shape(tvecs))

# undistort
dst = cv2.undistort(img, mtx,dist,None,mtx)

cv2.imwrite('calibresult.png',dst)



Camera matrix : 

[[2.22339353e+03 0.00000000e+00 1.02888550e+03]
 [0.00000000e+00 2.22295242e+03 1.04050529e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 

[[-0.12851724  0.10091855 -0.00062626  0.00043284 -0.02310165]]
size of dist： (1, 5)
rvecs : 

[array([[0.36036209],
       [0.34944765],
       [3.05506794]]), array([[ 0.68579551],
       [-0.24319838],
       [-3.1275911 ]]), array([[1.43613429],
       [0.38222864],
       [2.79068071]]), array([[ 0.17892567],
       [ 0.3679995 ],
       [-3.02543938]]), array([[-0.38074544],
       [ 0.2590631 ],
       [ 3.00858366]]), array([[0.63737436],
       [0.22716007],
       [3.04016189]]), array([[-0.00627764],
       [ 0.2345329 ],
       [-3.08473257]]), array([[-0.69833851],
       [ 0.2410319 ],
       [ 2.93799538]]), array([[-1.46052902],
       [-0.35512217],
       [-2.61390573]]), array([[-1.19888383],
       [-1.17461291],
       [-2.35898451]]), array([[ 0.6530366 ],
       [-0.11034322],
       [ 2.844690

True

In [2]:
userinput = 3
r = cv2.Rodrigues(rvecs[userinput])
print(np.concatenate((r[0],tvecs[userinput]),axis =1))

[[-0.98922093  0.10179358 -0.10526164  3.40168709]
 [-0.07359173 -0.96707535 -0.24361757  4.80626248]
 [-0.12659464 -0.23324522  0.96414235 18.21849246]]
